In [21]:
import pandas as pd
import numpy as np

First, I check the structure of the "xls" file, and I could determine which sheets I can to use for the analysis.

In [22]:
xls = pd.ExcelFile("FoodCompAfricaWest.xlsx")
food = pd.read_excel(xls, 2)

I eliminate the "\n" in the columns' name


In [23]:
columns = food.columns.to_list()
columns = [element.replace('\n', ' ').upper() for element in columns]
food.columns = columns

I will eliminate the rows with a number of N/A values greater than 44 also I will eliminate the first two rows.

In [24]:
condition = food.isna().sum(axis=1) > 44
food.drop(food[condition].index, inplace=True)
food.drop(index=[0,1], inplace=True)

In own dataframe in the last column exist diferent types of data so I handle the data and I eliminated the "[]", and I maked new data for a new column that will especify the method by wich the creators of the table found the phytatic acid.

In [26]:
columns = food.columns
columns = columns[5:]

for col in columns:
    lista = food[col].to_list() # Data of the last column of Food df
    copy_lista = lista[:]
    i = 0

    for num in copy_lista:

        try:
            if num.find("[") != -1:
                
                num = num.replace("[", "")
                num = num.replace("]", "")
                lista[i] = num

        except AttributeError:
            pass

        i += 1
    

I change the data type of each column

In [27]:
for col in columns:
    food[col] = pd.to_numeric(food[col], errors="coerce")

Now, I will find the food group's names

In [28]:
condition_sep = food.isna().sum(axis=1) > 43
group_index = food[condition_sep].index.to_list()

**Version 1**

I will create the diferents dataframe of the groups of food
The name of the list of dataframe groups is group_food

**Version 2**

I created a new column named group



In [29]:
# this cell have to be using if we want separated food groups
group_food = [] # Save the data
copy = group_index[:] # Ward for handling data
food["GROUP"] = food["CODE"]

try:
    
    for index in group_index:
        iterador = iter(copy)
        copy.pop(0) # eliminate the first element of copy in each iteration
        next_index = next(iterador)
        begin = index
        end = next_index - 2
        # I use 2 because previously I erased the row
       
        food.loc[begin: end, ("GROUP")] = food.loc[begin, ("CODE")].split("/")[0]
       
        
        
except StopIteration:
    begin = group_index[-1]
    end = food.iloc[-1].name
   
    food.loc[begin: end, ("GROUP")] = food.loc[begin, ("CODE")].split("/")[0]
    

Eliminated rows with the name of groups

In [33]:
for index in group_index:
    food.drop(index, inplace=True)

In [34]:
food

,CODE,FOOD NAME IN ENGLISH,FOOD NAME IN FRENCH,SCIENTIFIC NAME,BIBLIOID/SOURCE,EDIBLE PORTION COEFFICIENT 1 (FROM AS PURCHASED TO AS DESCRIBED),ENERGY (KJ),ENERGY (KCAL),WATER (G),"PROTEIN, TOTAL (G)",...,VITAMIN B12 (MCG),VITAMIN C (MG),CHOLESTEROL (MG),"FATTY ACIDS, TOTAL SATURATED (G)","FATTY ACIDS, TOTAL MONOUNSATURATED (G)","FATTY ACIDS, TOTAL POLYUNSATURATED (G)",LINOLEIC ACID (G),ALPHA-LINOLENIC ACID (G),"PHYTATE, TOTAL OR [PHYTATE, DETERMINED BY DIRECT PRECIPITATION] OR [PHYTATE, DETERMINED BY INDIRECT PRECIPITATION] (MG)",GROUP
3,01_172,Baling béinré (northern Burkina Faso)*: sorghu...,Baling béinré (nord du Burkina Faso)*: bouilli...,NaN,calc. from recipe / de recette,NaN,329.0,78.0,80.1,2.5,...,0.12,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Cereals and their products
4,01_173,Baling ni zièm béinré (Burkina Faso)*: porridg...,Baling ni zièm béinré (Burkina Faso)*: bouilli...,NaN,calc. from recipe / de recette,NaN,277.0,65.0,83.3,0.9,...,0.00,0.0,0.0,0.03,0.04,0.04,0.04,0.00,21.0,Cereals and their products
5,01_168,Beenkida (Burkina Faso)*: maize porridge with ...,Beenkida (Burkina Faso)*: bouillie de maïs ave...,NaN,calc. from recipe / de recette,NaN,426.0,100.0,74.6,2.1,...,0.00,0.0,0.0,0.05,0.12,0.13,0.12,0.00,41.0,Cereals and their products
6,01_188,"Biscuit, sweet, plain, unfortified","Biscuit, sucré, nature, non enrichi",NaN,"AU14(02C20287; 02C20364; 02C20286; 02C20314), ...",1.0,2010.0,479.0,4.6,6.2,...,0.06,1.0,30.0,10.30,7.65,2.38,2.03,0.29,NaN,Cereals and their products
7,01_177,Boalboal (Burkina Faso)*: millet balls with fr...,Boalboal (Burkina Faso)*: boulettes de mil ave...,NaN,calc. from recipe / de recette,NaN,1050.0,252.0,54.7,5.4,...,1.20,6.0,8.0,2.65,5.11,8.08,7.81,0.10,NaN,Cereals and their products
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,14_024,Téi né maane zéindo (Burkina Faso)*: oil palm ...,Téi né maane zéindo (Burkina Faso)*: noix de p...,NaN,calc. from recipe / de recette,NaN,399.0,96.0,83.5,2.7,...,0.95,3.0,6.0,2.77,2.83,0.80,0.09,0.01,NaN,Soups and sauces
1053,14_023,Téi zéindo (Burkina Faso)*: simple oil palm fr...,Téi zéindo (Burkina Faso)*: sauce simple de no...,NaN,calc. from recipe / de recette,NaN,823.0,199.0,71.5,1.7,...,0.42,4.0,3.0,6.47,6.85,1.69,0.18,0.01,NaN,Soups and sauces
1054,14_020,Toeg koing zéédo (Burkina Faso)*: dried baobab...,Toeg koing zéédo (Burkina Faso)*: sauce de feu...,NaN,calc. from recipe / de recette,NaN,181.0,44.0,89.1,2.8,...,NaN,5.0,5.0,0.50,0.69,1.09,0.95,0.04,NaN,Soups and sauces
1055,14_022,Toeg maas zéédo (Burkina Faso)*: fresh baobab ...,Toeg maas zéédo (Burkina Faso)*: sauce de feui...,NaN,calc. from recipe / de recette,NaN,251.0,60.0,86.7,3.1,...,NaN,9.0,6.0,0.74,1.20,1.87,1.71,0.04,NaN,Soups and sauces


In [36]:
food.to_csv("comida.csv")